In [1]:
import spacy
nlp = spacy.load('en')

In [51]:
s1 = 'The Magnificant Seven is a remake movie of Seven Samurai created in 2016.'
s2 = ''
s3 = ''
s4 = 'Singham is a remake of south indian movie Police.'



In [52]:
sentence = s4
parse(sentence)

Matching with Remake Template for: Singham is a remake of south indian movie Police.
has valid noun :  True
--------- Extracted Information -----------
Movie name :Singham
Original movie name : south indian movie Police
Time: <no-match>


In [50]:
def parse(statement1):
    isValidForRemakeTemplate = checkRemakeTemplateEligibility(statement1)
    #print(isValidForRemakeTemplate)
    if(isValidForRemakeTemplate):

        print("--------- Extracted Information -----------")
        
        movieName = extractMovieName(statement1)
        print("Movie name :" + str(movieName))

        originaMovieName = extractOriginalMovieName(statement1)
        print("Original movie name :" , originaMovieName)

        year = extractYear(statement1)
        print("Time:" , year)  
    else:
        print(" NO - PARSE ")

In [29]:
statement1 = 'The Magnificant Seven is a remake movie of Seven Samurai created in 2016.'

isValidForRemakeTemplate = checkRemakeTemplateEligibility(statement1)
print(isValidForRemakeTemplate)
if(isValidForRemakeTemplate):
    
    movieName = extractMovieName(statement1)
    print("extracted movie name is:" + str(movieName))
    
    originaMovieName = extractOriginalMovieName(statement1)
    print("extracted original movie name is:" , originaMovieName)
    
    year = extractYear(statement1)
    print("extracted year is:" , year)  

Matching with Remake Template for: The Magnificant Seven is a remake movie of Seven Samurai created in 2016.
has valid noun :  True
True
extracted movie name is:The Magnificant Seven
Samurai
extracted original movie name is: Seven Samurai
extracted year is: 2016


In [3]:
def extractMovieName(statement):
    doc = nlp(statement)
    
    movieName = "<unintialized>"
    
    for token in doc:
        if (token.dep_ == "nsubj" and token.head.pos_ == "VERB"):
            movieName = getNounChunkThatContainsNoun(statement, token)
            return movieName
        
    return "<no-match>"

In [38]:
def extractOriginalMovieName(statement):
    doc = nlp(statement)
    
    originalMovieName = "<uninititalized>"
    
    for token in doc:
        #if (token.dep_ == "compound"):
            #print("")
        if (token.dep_ == "pobj"):
            #print(token)
            originalMovieName = getNounChunkThatContainsNoun(statement, token)
            return originalMovieName
    
    return "<no-match>"

In [5]:
def extractYear(statement):
    doc = nlp(statement)
    for ent in doc.ents:
        if (ent.label_ == "DATE" or ent.label_ == "TIME"):
            return ent.text
    
    return "<no-match>"

In [6]:
def extractRecognizerName(statement):
    targetVerbLemmas = ['nominate','win', 'award']
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    writerName = "<uninitialized>"
    
#     handle when release is with premiered.
    while verbTextDNode[1] == 'conj':
        verbText = verbTextDNode[2]
        print("verbTextChanged to"+verbText)
        verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)

    print("vt:"+verbText)

    writerName = getWriter(statement,verbText)
    
    return writerName

In [7]:
def extractRecognizedAs(statement):
    targetVerbLemmas = ['nominate','win', 'award']
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    recognizedAs = "<uninitialized>"
    
    print("vt:"+verbText)

    recognizedAs = getRecognizedAs(statement,verbText)
    
    return recognizedAs

In [8]:
def getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas):
    """
    this will reutrn exact apperance of verb in statement.
    ex: statement = "Movie was written by James.", targetVerbLemmas = ['write','script']
    this will return "written" as it's lemma matches with one of the targetVerbLemmas
    """
    verbTokens = filterVerbTokens(statement)
    for token in verbTokens:
        if token.lemma_ in targetVerbLemmas:
            return token.text
    return "<no-match>"

In [9]:
def getNounSubject(statement, verbText):
    doc = nlp(statement)
    for token in doc:
        print(token.text, token.dep_, token.head.text)
        if((token.dep_ == "nsubjpass" or token.dep_ == "nsubj") and token.head.text == verbText):
            return token.text
    return "<no-match>"

In [10]:
def getMovie(statement, verbText):
    doc = nlp(statement)
    theme = "<uninitialized>" #movie
    
    for token in doc:
        #print(token.text, token.dep_, token.head.text)        
        if (token.dep_ == "nsubjpass" and token.head.text == verbText):
            print("token nsubjpass : " + token.text)
            theme = getNounChunkThatContainsNoun(statement, token)
            return theme
        if (token.dep_ == "nsubj" and token.head.text == verbText):
            print("token nsubj : " + token.text)
            theme = getNounChunkThatContainsNoun(statement, token)
            return theme
        elif (token.dep_ == "dobj" and token.head.text == verbText):
            theme = getNounChunkThatContainsNoun(statement, token)
            return theme
        elif ((token.dep_ == "conj" and token.head.text == verbText) or (token.dep_ == "conj" and token.text == verbText)):
            print("token conj : " + token.text)
            if (token.head.dep_ == "ROOT"):
                print("token.head: " + token.head.text)
                token1 = token.head
                return getMovie(statement, token1.text)
                '''
                if (token.dep_ == "nsubjpass" and token.head.text == token1.text):
                    theme = getNounChunkThatContainsNoun(statement, token)
                    return theme
                '''
    
    return "<no-match>"

In [11]:
def getWriter(statement, verbText):
    doc = nlp(statement)
    agent = "<uninitialized>" #writer
    '''
    for token in doc:
        #print(token.text, token.dep_, token.head.text)        
        if (token.dep_ == "agent" and token.head.text == verbText):
            token = [child for child in token.children][0]
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
        elif (token.dep_ == "nsubj" and token.head.text == verbText):
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
    '''
    
    for token in doc:
        #print("token:",token)
        '''
        if (token.dep_ == "agent" and token.head.text == verbText):
            token = [child for child in token.children][0]
            agent = getNounChunkThatContainsNoun(statement, token)
            print("Agent (agent) : " + agent)
            return agent
        '''
        if (token.dep_ == "nsubj" and token.head.text == verbText):
            agent = getNounChunkThatContainsNoun(statement, token)
            print("Agent (nsubj) : " + agent)
            return agent
        
        elif((token.dep_ == "conj" and token.head.text == verbText) or (token.dep_ == "conj" and token.text == verbText)):
            agent = extractOwnerEntitySyntactically(statement)
            return agent
        
        elif(token.head.text == verbText or token.text == verbText):
            # agent = extractOwnerEntitySyntactically(statement)
            agent = getByEntity(statement)
            return agent
        
    return agent

In [12]:
def getByEntity(statement):
    doc = nlp(statement)
    
    for token in doc:
        if token.text == "by":
            for child in token.children:
                if (getORGorPersonThatContainsToken(statement, child) != "<no-match>"):
                    return getORGorPersonThatContainsToken(statement, child)
    return "<no-match>"

In [13]:
def extractOwnerEntitySyntactically(statement):
    doc = nlp(statement)
    verbTokens = filterVerbTokens(statement)
    ORGEntity = []
    
    for token in verbTokens:
        if isExpectedVerbToken(token):
            #print("Expected token is:"+token.text)
            #expected children [was, theatrically, in, on, by]
            byToken = None
            for child in token.children: 
                #usually release location starts with "in"
                if child.text == "by":
                    byToken = child
             
            #print("bytoken : ->", byToken)
            if byToken == None:
                #means no direct by available so lets look at some conj child
                for child in token.children: 
                    #usually release location starts with "in"
                    if child.dep_ == "conj":
                        for grandChild in child.children:
                            if grandChild.text == "by":
                                byToken = grandChild
            
            if byToken != None:
                for subChild in byToken.children:
                    ORGEntity.append(getORGorPersonThatContainsToken(statement, subChild))
                    #print("subchild : " + subChild.text)
                    if (subChild.dep_ == "pobj" and subChild.head.text == "by"):
                        grandChildren = subChild.children
                        childEntity = getORGEntityForGrandChild1(statement, grandChildren, ORGEntity)
                        #print("child entity:    " , str(childEntity))
                        return list(set(childEntity))
                        #ORGEntity.clear()
                        #for item in childEntity:
                            #ORGEntity.append(item)
                    return ORGEntity
    return "<no-match>"

In [14]:
def getORGEntityForGrandChild1(statement, children, ORGEntity):
    '''
    New definition
    '''
    doc = nlp(statement)
    
    for child in children:
        sizeChild = sum(1 for item in child.children)
        print(child, sizeChild)
        if sizeChild > 0:
            ORGEntity.append(getORGorPersonThatContainsToken(statement, child))
            print(ORGEntity)
            #ORGEntity.append(getORGEntityForGrandChild1(statement, child.children, ORGEntity))
            getORGEntityForGrandChild1(statement, child.children, ORGEntity)
        if sizeChild == 0:
            if (getORGorPersonThatContainsToken(statement, child) != "<no-match>"):
                ORGEntity.append(getORGorPersonThatContainsToken(statement, child))
    
    return ORGEntity

In [15]:
def isExpectedVerbToken(token):
    return (token.lemma_ in ["nominate"] or token.lemma_ in ["win"] or token.lemma_ in ["award"])

In [16]:
def getORGorPersonThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for ent in doc.ents:
        if(token.text in ent.text and tokenStartPos >= ent.start_char and tokenStartPos <= ent.end_char):
            return ent.text
    return "<no-match>"

In [17]:
import re
def calculateStartPositionOfToken(statement, targetToken):
    """
    this should return start position of the token
    """
    matches = [m.start() for m in re.finditer(targetToken.text, statement)]
    if(len(matches)==1):
        return matches[0]
    

    huristicPosition = 0
    doc = nlp(statement)
    
    i = 0
    for token in doc:
        if i < targetToken.i:
            huristicPosition = huristicPosition + len(token.text)+1
            i = i+1
        else:
            break
        
    distances = []
    for match in matches:
        distances.append(abs(match-huristicPosition))
        
    minIndex = distances.index(min(distances))
    
    return matches[minIndex]


In [18]:
def getNounChunkThatContainsNoun(statement, nounSubject):
    doc = nlp(statement)
    for chunk in doc.noun_chunks:
        if nounSubject.text in chunk.text:
            return chunk.text
    return "<no-match>"

In [19]:
def checkRemakeTemplateEligibility(statement):
    print("Matching with Remake Template for: " + statement)
    hasValidToken = check_word_pattern(statement, " remake ")
    print("has valid noun : " , hasValidToken)
    
    return hasValidToken

In [20]:
def check_word_pattern(statement, pattern1):
    return ((pattern1 in statement))

In [21]:
def checkRecognizedAsTemplateEligibility(statement):
    print("Matching with wriiten template: for " + statement)
    hasValidVerb = check_verb_match(statement)
    print("Has valid verb:" + str(hasValidVerb))
    
    return hasValidVerb

In [22]:
def check_verb_match(statement):
    verbTokens = filterVerbTokens(statement)
    targetVerbs = ['nominate','win', 'award']
    for token in verbTokens:
        if token.lemma_ in targetVerbs:
            return True
    return False

In [23]:
def filterVerbTokens(statement):
    doc = nlp(statement)
    result = list(filter(lambda token: token.pos_ == "VERB" and token.lemma_ != "be", doc))
    return result

In [24]:
def get_dependency_tree_nodes(sentence):
    nodes = []
    doc = nlp(sentence)
    for token in doc:
        nodes.append([token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children]])
    return nodes

In [25]:
get_dependency_tree_nodes('The Mission Impossible was nominated for Golden Globe Award.')

[['The', 'det', 'Impossible', 'PROPN', []],
 ['Mission', 'compound', 'Impossible', 'PROPN', []],
 ['Impossible', 'nsubjpass', 'nominated', 'VERB', [The, Mission]],
 ['was', 'auxpass', 'nominated', 'VERB', []],
 ['nominated', 'ROOT', 'nominated', 'VERB', [Impossible, was, for, .]],
 ['for', 'prep', 'nominated', 'VERB', [Award]],
 ['Golden', 'compound', 'Globe', 'PROPN', []],
 ['Globe', 'compound', 'Award', 'PROPN', [Golden]],
 ['Award', 'pobj', 'for', 'ADP', [Globe]],
 ['.', 'punct', 'nominated', 'VERB', []]]

In [26]:
get_dependency_tree_nodes('The Mission Impossible was awarded Golden Globe Award.')

[['The', 'det', 'Impossible', 'PROPN', []],
 ['Mission', 'compound', 'Impossible', 'PROPN', []],
 ['Impossible', 'nsubjpass', 'awarded', 'VERB', [The, Mission]],
 ['was', 'auxpass', 'awarded', 'VERB', []],
 ['awarded', 'ROOT', 'awarded', 'VERB', [Impossible, was, Award, .]],
 ['Golden', 'compound', 'Globe', 'PROPN', []],
 ['Globe', 'compound', 'Award', 'PROPN', [Golden]],
 ['Award', 'dobj', 'awarded', 'VERB', [Globe]],
 ['.', 'punct', 'awarded', 'VERB', []]]

In [27]:
get_dependency_tree_nodes('The Mission Impossible won Golden Globe.')

[['The', 'det', 'Impossible', 'PROPN', []],
 ['Mission', 'compound', 'Impossible', 'PROPN', []],
 ['Impossible', 'nsubj', 'won', 'VERB', [The, Mission]],
 ['won', 'ROOT', 'won', 'VERB', [Impossible, Globe, .]],
 ['Golden', 'compound', 'Globe', 'PROPN', []],
 ['Globe', 'dobj', 'won', 'VERB', [Golden]],
 ['.', 'punct', 'won', 'VERB', []]]

In [28]:
get_dependency_tree_nodes('The Magnificent Seven is a remake of Seven Samurai created in 2016.')

[['The', 'det', 'Seven', 'PROPN', []],
 ['Magnificent', 'compound', 'Seven', 'PROPN', []],
 ['Seven', 'nsubj', 'is', 'VERB', [The, Magnificent]],
 ['is', 'ROOT', 'is', 'VERB', [Seven, remake, .]],
 ['a', 'det', 'remake', 'NOUN', []],
 ['remake', 'attr', 'is', 'VERB', [a, of]],
 ['of', 'prep', 'remake', 'NOUN', [Samurai]],
 ['Seven', 'compound', 'Samurai', 'PROPN', []],
 ['Samurai', 'pobj', 'of', 'ADP', [Seven, created]],
 ['created', 'acl', 'Samurai', 'PROPN', [in]],
 ['in', 'prep', 'created', 'VERB', [2016]],
 ['2016', 'pobj', 'in', 'ADP', []],
 ['.', 'punct', 'is', 'VERB', []]]

In [15]:
doc = nlp("The Magnificent Seven is a remake movie of Seven Samurai created in 1984.")
for chunk in doc.noun_chunks:
    print(chunk)

The Magnificent Seven
a remake movie
Seven Samurai


In [14]:
doc = nlp ("The Magnificent Seven is a remake movie of Seven Samurai created in 2016.")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Seven 16 21 CARDINAL
Seven 43 48 CARDINAL
Samurai 49 56 ORG
2016 68 72 DATE
